In [1]:
!pip install langchain langchain-openai python-dotenv python-docx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00


In [2]:
from dotenv import load_dotenv
import os
from google.colab import userdata
os.environ["GMAIL_ADDRESS"]=userdata.get('GMAIL_ADDRESS')
os.environ["GMAIL_PASSWORD"]=userdata.get('GMAIL_PASSWORD')
# Load environment variables from a .env file
load_dotenv()
GMAIL_ADDRESS = os.getenv("GMAIL_ADDRESS")     # Your Gmail address
GMAIL_PASSWORD = os.getenv("GMAIL_PASSWORD")   # Your Gmail app password

if not GMAIL_ADDRESS or not GMAIL_PASSWORD:
    raise ValueError("Gmail address or password not found in environment variables.")

In [3]:
import imaplib
import smtplib

# Connect to Gmail IMAP over SSL
try:
    imap_client = imaplib.IMAP4_SSL('imap.gmail.com', 993)
    imap_client.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
    print("IMAP connection established.")
except Exception as e:
    print(f"Failed to connect to IMAP: {e}")

# Connect to Gmail SMTP over SSL
try:
    smtp_client = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    smtp_client.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
    print("SMTP connection established.")
except Exception as e:
    print(f"Failed to connect to SMTP: {e}")

IMAP connection established.
SMTP connection established.


In [4]:
from typing import Optional, List, Type
from langchain.callbacks.manager import (
    CallbackManagerForToolRun,
    AsyncCallbackManagerForToolRun
)
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# Define a Pydantic schema for input parameters
class EmailOperationInput(BaseModel):
    operation_type: str = Field(..., description="Operation to perform: send_email, fetch_last, fetch_last_with_attachment, fetch_by_sender, fetch_n, fetch_between, detect_chain")
    receiver_email: Optional[str] = Field(None, description="Recipient email address (for sending)")
    subject: Optional[str] = Field(None, description="Email subject (for sending)")
    body: Optional[str] = Field(None, description="Email body text (for sending)")
    attachment_path: Optional[str] = Field(None, description="Path to attachment file (for sending)")
    sender_email: Optional[str] = Field(None, description="Sender email address (for fetching by sender)")
    number_of_emails: Optional[int] = Field(None, description="Number of recent emails to fetch")
    start_date: Optional[str] = Field(None, description="Start date (e.g., '01-Jan-2023') for date range")
    end_date: Optional[str] = Field(None, description="End date (e.g., '31-Dec-2023') for date range")
    message_id: Optional[str] = Field(None, description="Message-ID header to detect reply chains")

In [8]:
import email

In [7]:
class EmailOperationsTool(BaseTool):
    """Tool to perform various email operations via Gmail (IMAP/SMTP)."""
    name: str = "email_operations"  # Added type annotation: str
    description: str = "Send or fetch Gmail emails, with support for attachments, date filtering, and thread detection."  # Added type annotation: str
    args_schema: Type[BaseModel] = EmailOperationInput

    def _run(self,
             operation_type: str,
             receiver_email: Optional[str] = None,
             subject: Optional[str] = None,
             body: Optional[str] = None,
             attachment_path: Optional[str] = None,
             sender_email: Optional[str] = None,
             number_of_emails: Optional[int] = None,
             start_date: Optional[str] = None,
             end_date: Optional[str] = None,
             message_id: Optional[str] = None,
             run_manager: Optional[CallbackManagerForToolRun] = None) -> str:
        """Execute the requested email operation."""
        # Helper function to prompt for missing input
        def prompt_if_missing(val, prompt_text):
            return val if val else input(prompt_text)

        # Ensure operation type is provided
        if not operation_type:
            operation_type = prompt_if_missing(operation_type, "Enter operation type: ")

        result = ""
        try:
            if operation_type == "send_email":
                # Prompt for missing send parameters
                receiver_email = prompt_if_missing(receiver_email, "Enter recipient email: ")
                subject = prompt_if_missing(subject, "Enter email subject: ")
                body = prompt_if_missing(body, "Enter email body: ")

                # Build the email message
                from email.message import EmailMessage
                msg = EmailMessage()
                msg["From"] = GMAIL_ADDRESS
                msg["To"] = receiver_email
                msg["Subject"] = subject
                msg.set_content(body)

                # Attach file if provided
                if attachment_path:
                    try:
                        with open(attachment_path, "rb") as fp:
                            data = fp.read()
                            maintype, subtype = "application", "octet-stream"
                            msg.add_attachment(data, maintype=maintype, subtype=subtype, filename=os.path.basename(attachment_path))
                            print(f"Attached file: {attachment_path}")
                    except Exception as e:
                        return f"Failed to attach file: {e}"

                # Send via SMTP
                smtp = smtplib.SMTP_SSL('smtp.gmail.com', 465)
                smtp.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                smtp.send_message(msg)
                smtp.quit()
                result = f"Email sent to {receiver_email}."

            elif operation_type == "fetch_last":
                # Fetch the last email from Inbox
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                _, data = imap.search(None, "ALL")
                id_list = data[0].split()
                if not id_list:
                    return "No emails found."
                latest_id = id_list[-1]
                _, msg_data = imap.fetch(latest_id, "(RFC822)")
                from email import policy
                msg = email.message_from_bytes(msg_data[0][1], policy=policy.default)
                result = f"From: {msg['From']}\nSubject: {msg['Subject']}\nDate: {msg['Date']}"
                imap.logout()

            elif operation_type == "fetch_last_with_attachment":
                # Fetch the last email that has an attachment
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                _, data = imap.search(None, "ALL")
                ids = data[0].split()[::-1]  # reverse (latest first)
                for eid in ids:
                    _, msg_data = imap.fetch(eid, "(RFC822)")
                    msg = email.message_from_bytes(msg_data[0][1], policy=email.policy.default)
                    if msg.is_multipart():
                        for part in msg.walk():
                            if part.get_content_disposition() == "attachment":
                                # Save attachment
                                filename = part.get_filename()
                                if filename:
                                    save_path = os.path.join("attachments", filename)
                                    os.makedirs(os.path.dirname(save_path), exist_ok=True)
                                    with open(save_path, "wb") as f:
                                        f.write(part.get_payload(decode=True))
                                    result = f"Downloaded attachment: {save_path}"
                                    break
                        if result:
                            result = f"Email From: {msg['From']}, Subject: {msg['Subject']} with attachment saved."
                            break
                if not result:
                    result = "No email with attachment found."
                imap.logout()

            elif operation_type == "fetch_by_sender":
                sender_email = prompt_if_missing(sender_email, "Enter sender's email to filter by: ")
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                # Search by FROM
                criterion = f'FROM "{sender_email}"'
                _, data = imap.search(None, criterion)
                ids = data[0].split()
                emails = []
                for eid in ids:
                    _, msg_data = imap.fetch(eid, "(RFC822)")
                    msg = email.message_from_bytes(msg_data[0][1], policy=email.policy.default)
                    emails.append(f"{msg['Date']} | {msg['From']} | {msg['Subject']}")
                result = "\n".join(emails) if emails else f"No emails from {sender_email}."
                imap.logout()

            elif operation_type == "fetch_n":
                number_of_emails = number_of_emails or int(prompt_if_missing(number_of_emails, "Enter number of recent emails to fetch: "))
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                _, data = imap.search(None, "ALL")
                ids = data[0].split()
                selected_ids = ids[-number_of_emails:]
                emails = []
                for eid in selected_ids:
                    _, msg_data = imap.fetch(eid, "(RFC822)")
                    msg = email.message_from_bytes(msg_data[0][1], policy=email.policy.default)
                    emails.append(f"{msg['Date']} | {msg['From']} | {msg['Subject']}")
                result = "\n".join(emails) if emails else "No emails found."
                imap.logout()

            elif operation_type == "fetch_between":
                # Fetch emails between specific dates (IMAP since/before)
                start_date = prompt_if_missing(start_date, "Enter start date (e.g. 01-Jan-2023): ")
                end_date = prompt_if_missing(end_date, "Enter end date (e.g. 31-Dec-2023): ")
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                # IMAP date format: DD-Mon-YYYY
                criterion = f'(SINCE "{start_date}" BEFORE "{end_date}")'
                _, data = imap.search(None, criterion)
                ids = data[0].split()
                emails = []
                for eid in ids:
                    _, msg_data = imap.fetch(eid, "(RFC822)")
                    msg = email.message_from_bytes(msg_data[0][1], policy=email.policy.default)
                    emails.append(f"{msg['Date']} | {msg['From']} | {msg['Subject']}")
                result = "\n".join(emails) if emails else f"No emails between {start_date} and {end_date}."
                imap.logout()

            elif operation_type == "detect_chain":
                message_id = prompt_if_missing(message_id, "Enter the Message-ID to trace reply chain: ")
                imap = imaplib.IMAP4_SSL('imap.gmail.com', 993)
                imap.login(GMAIL_ADDRESS, GMAIL_PASSWORD)
                imap.select("INBOX")
                # Search for emails that have the given ID in In-Reply-To header
                criterion = f'HEADER In-Reply-To "{message_id}"'
                _, data = imap.search(None, criterion)
                ids = data[0].split()
                if not ids:
                    result = f"No replies found for Message-ID {message_id}."
                else:
                    threads = []
                    for eid in ids:
                        _, msg_data = imap.fetch(eid, "(RFC822)")
                        msg = email.message_from_bytes(msg_data[0][1], policy=email.policy.default)
                        threads.append(f"{msg['Date']} | {msg['From']} -> {msg['Subject']}")
                    result = "Reply chain:\n" + "\n".join(threads)
                imap.logout()

            else:
                result = f"Unknown operation: {operation_type}"

        except Exception as e:
            result = f"Error during '{operation_type}': {e}"

        return result

    async def _arun(self, *args, **kwargs):
        """Async not implemented."""
        raise NotImplementedError("EmailOperationsTool does not support async operation.")

In [9]:
tool = EmailOperationsTool()

In [10]:
response = tool._run(
    operation_type="send_email",
    receiver_email="dhruvsh1997@gmail.com",
    subject="Meeting Reminder",
    body="Don't forget our meeting tomorrow at 10 AM.",
    attachment_path=None  # Or path to a file to attach
)
print(response)

Email sent to dhruvsh1997@gmail.com.


In [16]:
response = tool._run(
    operation_type="send_email",
    receiver_email="dhruvsh1997@gmail.com",
    subject="Meeting Agenda",
    body="Please find tommorrow's Meeting Agenda Document.",
    attachment_path="/content/traffic.pdf"  # Or path to a file to attach
)
print(response)

Attached file: /content/traffic.pdf
Email sent to dhruvsh1997@gmail.com.


In [11]:
print(tool._run(operation_type="fetch_last"))

From: Medium Daily Digest <noreply@medium.com>
Subject: Physics-Informed Neural Networks for Anomaly Detection: A Practitioner’s Guide | Shuai Guo, PhD
Date: Sun, 27 Apr 2025 01:50:00 +0000


In [12]:
print(tool._run(operation_type="fetch_last_with_attachment"))

Email From: Dhruv Sharma <dhruvsh1997@gmail.com>, Subject: Transformers Code with attachment saved.


In [13]:
print(tool._run(operation_type="fetch_by_sender", sender_email="dhruvsh1997@gmail.com"))

Wed, 31 Jul 2024 18:36:10 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Y
Mon, 11 Nov 2024 19:12:58 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Response
Mon, 11 Nov 2024 19:14:09 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | YRL
Tue, 12 Nov 2024 16:09:17 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | JK
Sat, 22 Feb 2025 21:28:33 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Fwd: Your Account is in Safe Mode
Sat, 22 Feb 2025 21:30:35 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Re: Your Account is in Safe Mode
Sun, 23 Feb 2025 22:46:58 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Fwd: Action advised: Dhruv, your Dropbox account is over capacity
Sun, 23 Feb 2025 22:56:28 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Re: Action advised: Dhruv, your Dropbox account is over capacity
Thu, 13 Mar 2025 17:29:12 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Re: EHO Optimizer Email
Mon, 31 Mar 2025 13:00:34 +0530 | Dhruv Sharma <dhruvsh1997@gmail.com> | Transformers Code
Mon, 14 Apr 

In [14]:
print(tool._run(operation_type="fetch_n", number_of_emails=3))#last 3 emails

Fri, 25 Apr 2025 15:23:33 +0000 | Grammarly <hello@mail.grammarly.com> | Cheers to  1 Year  with Grammarly
Sat, 26 Apr 2025 01:50:00 +0000 | Medium Daily Digest <noreply@medium.com> | Image Captioning, Transformer Mode On | Muhammad Ardi in Data Science Collective
Sun, 27 Apr 2025 01:50:00 +0000 | Medium Daily Digest <noreply@medium.com> | Physics-Informed Neural Networks for Anomaly Detection: A Practitioner’s Guide | Shuai Guo, PhD


In [15]:
print(tool._run(operation_type="detect_chain", message_id="<dhruvsh1997@gmail.com>"))

No replies found for Message-ID <dhruvsh1997@gmail.com>.
